In [17]:
import gurobipy as gp
from gurobipy import GRB

# Data
nodes = [0, 1, 2]  # Nodes
b = [100, 200, 1000]  # Demands for each node
max_diff = 50  # Maximum allowed difference in slack or x
big_M = 10000  # Big M penalty

# Create the model
model = gp.Model("Fairness")

# Decision Variables
x = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="x")  # Allocation variables
delta_plus = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="delta_plus")  # Positive slack
delta_minus = model.addVars(nodes, vtype=GRB.CONTINUOUS, name="delta_minus")  # Negative slack

# Auxiliary Variables for absolute value linearization
z = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, name="z")  # For absolute differences
z_x = model.addVars(nodes, nodes, vtype=GRB.CONTINUOUS, name="z_x")  # For absolute differences in x

# Objective: Maximize allocation minus penalized slack
model.setObjective(
    gp.quicksum(x[i] for i in nodes) - big_M * gp.quicksum(delta_plus[i] + delta_minus[i] for i in nodes),
    GRB.MAXIMIZE
)

# Constraints
for i in nodes:
    # Slack constraints
    model.addConstr(x[i] + delta_plus[i] - delta_minus[i] == b[i], name=f"slack_{i}")

# Absolute value difference constraints for slack
for i in nodes:
    for j in nodes:
        if i < j:
            # Linearize the absolute value for slack differences
            model.addConstr((delta_plus[i] + delta_minus[i]) - (delta_plus[j] + delta_minus[j]) <= z[i, j], name=f"diff_pos_{i}_{j}")
            model.addConstr((delta_plus[j] + delta_minus[j]) - (delta_plus[i] + delta_minus[i]) <= z[i, j], name=f"diff_neg_{i}_{j}")
            # Ensure the absolute difference is within the maximum allowed range
            model.addConstr(z[i, j] <= max_diff, name=f"max_diff_{i}_{j}")

# Absolute value difference constraints for x
for i in nodes:
    for j in nodes:
        if i < j:
            # Linearize the absolute value for x differences
            model.addConstr(x[i] - x[j] <= z_x[i, j], name=f"x_diff_pos_{i}_{j}")
            model.addConstr(x[j] - x[i] <= z_x[i, j], name=f"x_diff_neg_{i}_{j}")
            # Ensure the absolute difference is within the maximum allowed range
            model.addConstr(z_x[i, j] <= max_diff, name=f"x_max_diff_{i}_{j}")

# Solve the model
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    print(f"Optimal value of the objective: {model.objVal}")
    for i in nodes:
        print(f"x[{i}] = {x[i].X}, delta_plus[{i}] = {delta_plus[i].X}, delta_minus[{i}] = {delta_minus[i].X}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 27 columns and 63 nonzeros
Model fingerprint: 0x16a79889
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+03]
Presolve removed 12 rows and 18 columns
Presolve time: 0.01s
Presolved: 9 rows, 15 columns, 33 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   3.000000e+30   3.000000e+00      0s
       7   -1.2248325e+07   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.224832500e+07
Optimal value of the objective: -12248325.0
x[0] = 525.0, delta_plus[0] = 0.0, delta_minus[0] = 425.0
x[1] = 575.0, delta_plus[1] = 0.0, delta_minus[1] = 375.0
x[2] = 575.0, delta